In [1]:
import pandas as pd

In [2]:
df_origin = pd.read_csv('./data/beers.csv').drop('Unnamed: 0', axis=1)

In [3]:
len(df_origin)

2410

In [4]:
df_origin.head()

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


In [5]:
df_origin.describe()

,abv,ibu,id,brewery_id,ounces
count,2348.000000,1405.000000,2410.000000,2410.000000,2410.000000
mean,0.059773,42.713167,1431.113278,231.749793,13.592241
std,0.013542,25.954066,752.459975,157.685604,2.352204
min,0.001000,4.000000,1.000000,0.000000,8.400000
25%,0.050000,21.000000,808.250000,93.000000,12.000000
50%,0.056000,35.000000,1453.500000,205.000000,12.000000
75%,0.067000,64.000000,2075.750000,366.000000,16.000000
max,0.128000,138.000000,2692.000000,557.000000,32.000000


In [6]:
df = df_origin.dropna(how='any')

In [7]:
df.head()

,abv,ibu,id,name,style,brewery_id,ounces
14,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0
21,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4
22,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0
24,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0
25,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0


In [8]:
df.describe()

,abv,ibu,id,brewery_id,ounces
count,1403.000000,1403.000000,1403.000000,1403.000000,1403.000000
mean,0.059919,42.739843,1413.888810,223.375624,13.510264
std,0.013585,25.962692,757.572191,150.387510,2.254112
min,0.027000,4.000000,1.000000,0.000000,8.400000
25%,0.050000,21.000000,771.000000,95.500000,12.000000
50%,0.057000,35.000000,1435.000000,198.000000,12.000000
75%,0.068000,64.000000,2068.500000,350.000000,16.000000
max,0.125000,138.000000,2692.000000,546.000000,32.000000


In [9]:
len(df['name'].unique())

1326

In [10]:
import util

In [11]:
name_dict = util.get_unique_dict(df['name'])
df_name = util.get_df(df['name'], name_dict, 'name')
df_name.index = df['name'].index

In [12]:
len(name_dict)

1326

In [13]:
style_dict = util.get_unique_dict(df['style'])
df_style = util.get_df(df['style'], style_dict, 'style')
df_style.index = df['style'].index

In [14]:
len(style_dict)

90

In [15]:
df_style.head()

,style
14,0
21,1
22,2
24,0
25,3


In [16]:
df_learn = pd.concat([
    df['abv'],
    df['ibu'],
    df_name['name'],
    df_style['style'],
], axis=1, sort=True)

In [17]:
df_learn.head()

,abv,ibu,name,style
14,0.061,60.0,0,0
21,0.099,92.0,1,1
22,0.079,45.0,2,2
24,0.044,42.0,3,0
25,0.049,17.0,4,3


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNC

In [19]:
def get_neigh(X, y):
    (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.1, random_state=3)
    neigh = KNC(n_neighbors=5)
    return neigh.fit(X_train, y_train), neigh.score(X_test, y_test)

Predict style from abv and ibu

In [20]:
X_style = df_learn.drop(['name', 'style'], axis=1).values
y_style = df_learn['style'].values

In [21]:
model, score = get_neigh(X_style, y_style)

In [22]:
score

0.40425531914893614

In [23]:
abv = 0.075
ibu = 45.6
pred = model.predict([
    (abv, ibu)
])
style_idx = pred[0]
print(style_idx)

0


In [24]:
style_name = util.get_unique_value(style_idx, style_dict)
print(style_name)

American Pale Ale (APA)


Predict name from abv, ibu and style

In [25]:
X_name = df_learn.drop(['name'], axis=1).values
y_name = df_learn['name'].values

In [26]:
model_n, score_n = get_neigh(X_name, y_name)

In [27]:
score_n

0.04964539007092199

In [28]:
style = style_idx
pred = model_n.predict([
    (abv, ibu, style)
])
name_idx= pred[0]
print(name_idx)

1057


In [29]:
name = util.get_unique_value(name_idx, name_dict)
print(name)

Pine Belt Pale Ale


In [30]:
beer = util.show_recommemd_beer(pred, name_dict, style_dict, df)
print(beer)

Pandas(Index=1954, abv=0.065, ibu=45.0, id=2387, name='Pine Belt Pale Ale', style='American Pale Ale (APA)', brewery_id=118, ounces=12.0)
style_id 0
Pandas(Index=1966, abv=0.065, ibu=45.0, id=45, name='Pine Belt Pale Ale', style='American Pale Ale (APA)', brewery_id=118, ounces=16.0)
style_id 0
None


In [31]:
for row in df.itertuples():
    if row.style == style_name:
        print(row.abv, row.ibu, row.name)

0.061 60.0 Bitter Bitch
0.044 42.0 Bitter American
0.044 42.0 Bitter American (2011)
0.042 42.0 A Beer
0.05 35.0 Oatmeal PSA
0.053 36.0 Klickitat Pale Ale
0.053 40.0 Freeride APA
0.055 28.0 Poleeko Gold Pale Ale (2009)
0.06 55.0 Sky High Rye
0.06 43.0 Naked Pig Pale Ale
0.044 38.0 Field 41 Pale Ale
0.049 28.0 Pamola Xtra Pale Ale
0.058 45.0 Fenway American Pale Ale
0.053 20.0 Steel Rail Extra Pale Ale
0.053 43.0 Jack Pine Savage
0.055 45.0 Jalapeno Pale Ale
0.058 44.0 Aftermath Pale Ale
0.046 35.0 834 Happy As Ale
0.059 60.0 Full Nelson Pale Ale
0.059 60.0 Full Nelson Pale Ale (2010)
0.053 45.0 Kindler Pale Ale
0.049 35.0 Hazed & Infused
0.057 35.0 Hoopla Pale Ale
0.049 35.0 Hazed & Infused (2010)
0.052000000000000005 17.0 Polar Pale Ale
0.055 40.0 Pale Alement
0.055 40.0 Pale Alement
0.056 36.0 Mucho Aloha Hawaiian Pale Ale
0.056 55.0 Caldera Pale Ale
0.056 55.0 Caldera Pale Ale (2010)
0.056 55.0 Caldera Pale Ale (2009)
0.056 55.0 Caldera Pale Ale (2005)
0.056 55.0 Caldera Pale Ale (2